### Multiclass Classification

In [1]:
from tables import *
import pandas as pd
import numpy as np
import tensorflow as tf

c:\python\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Load data

In [2]:
train = pd.read_hdf("train.h5", "train")
test = pd.read_hdf("test.h5", "test")

#View training data
#train.shape (45324, 101)
test.index

Int64Index([45324, 45325, 45326, 45327, 45328, 45329, 45330, 45331, 45332,
            45333,
            ...
            53451, 53452, 53453, 53454, 53455, 53456, 53457, 53458, 53459,
            53460],
           dtype='int64', length=8137)

In [3]:
y_train = train['y']
x_train = train._drop_axis(['y'], axis=1)

#Switch to numpy
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(test)

In [4]:
num_features = len(x_train[1])
total = 0

#standardize data, so for each x1..x100, do (x-mean)/sd(x) s.t. mean(x) ~= 0
for i in range(num_features):
    x_train[:,i] = (x_train[:,i] - np.mean(x_train[:,i])) / np.std(x_train[:,i])
    x_test[:,i] = (x_test[:,i] - np.mean(x_test[:,i])) / np.std(x_test[:,i])
    total += np.mean(x_train[:,i])


#adding all the means, we still have a very small # close to 0
total

-0.00011446602163711628

## KERAS ATTEMPT

In [5]:

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.regularizers import l2

lambdas = [.9, .3, .1, .03, .01, .005, .001]

for l in lambdas:
    
    #Define model
    model = Sequential([Dense(70, input_shape=(100,), W_regularizer=l2(l)), 
                       Activation('relu'),
                       Dense(30),
                       Activation('relu'),
                       Dense(50),
                       Activation('relu'),
                       Dense(20),
                       Activation('relu'),
                       Dense(5),
                       Activation('softmax'),
    ])
    
    model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

    #Easy way to convert to one-hot
    y_train_hot2 = keras.utils.to_categorical(y_train, num_classes=5)

    model.fit(x_train, y_train_hot2, epochs=100)
    
    pred_one_hot = model.predict(x_test)
    
    pred = []

    for line in pred_one_hot:
        pred.append(np.argmax(line))
        
    d = {'Id': test.index, 'y': pred}
    out = pd.DataFrame(d)
    out.to_csv('output_' + str(l) + '.csv', index=False)
    

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
c:\python\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(70, input_shape=(100,), kernel_regularizer=<keras.reg...)`
  if sys.path[0] == '':


Epoch 1/100
45324/45324 [==============================] - 2s 35us/step - loss: 3.7517 - acc: 0.7270
Epoch 2/100
45324/45324 [==============================] - 2s 48us/step - loss: 0.7528 - acc: 0.7894
Epoch 3/100
45324/45324 [==============================] - 4s 97us/step - loss: 0.7090 - acc: 0.7993
Epoch 4/100
45324/45324 [==============================] - 5s 110us/step - loss: 0.6882 - acc: 0.8033
Epoch 5/100
45324/45324 [==============================] - 5s 116us/step - loss: 0.6675 - acc: 0.8088
Epoch 6/100
45324/45324 [==============================] - 5s 114us/step - loss: 0.6572 - acc: 0.8125
Epoch 7/100
45324/45324 [==============================] - 5s 121us/step - loss: 0.6473 - acc: 0.8132
Epoch 8/100
45324/45324 [==============================] - 6s 124us/step - loss: 0.6365 - acc: 0.8181
Epoch 9/100
45324/45324 [==============================] - 6s 125us/step - loss: 0.6284 - acc: 0.8195
Epoch 10/100
45324/45324 [==============================] - 6s 126us/step - loss: 0.6

KeyboardInterrupt: 

In [ ]:
### Training
#model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

#Easy way to convert to one-hot
#y_train_hot2 = keras.utils.to_categorical(y_train, num_classes=5)

#model.fit(x_train, y_train_hot2, epochs=100)

In [ ]:
#pred_one_hot = model.predict(x_test)
#type(pred_one_hot)

In [ ]:
#Convert predictions back from 1-hot
#pred = []

#for line in pred_one_hot:
   # pred.append(np.argmax(line))

In [ ]:
#d = {'Id': test.index, 'y': pred}
#out = pd.DataFrame(d)
#out.to_csv('output.csv', index=False)

In [ ]:
#d = pd.DataFrame([1,3])

In [ ]:
#d.to_csv('output_' + '.5' + '.csv', index=False)

In [ ]:
#foo = 5

In [ ]:
#d.to_csv('output_' + foo + '.csv')

In [ ]:

#l = 5
#out.to_csv('output_' + str(l) + '.csv', index=False)